<a href="https://colab.research.google.com/github/afifhusman/DLCVn/blob/main/Solution_asg1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Muhammad Afif Bin Husman**

**Solution = Assignment 1**

## solution adapted from
[1] https://www.digitalocean.com/community/tutorials/how-to-detect-and-extract-faces-from-an-image-with-opencv-and-python

[2] https://stackoverflow.com/questions/24195138/gaussian-blurring-with-opencv-only-blurring-a-subregion-of-an-image


---





In [1]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

#function to take photo using JS

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
from IPython.display import Image
import cv2
import matplotlib.pyplot as plt
from random import choices
from string import digits

for x in range(0, 3):                                    #Loop 3 times
  
  try:
    
    f = 'photo' + "".join(choices(digits, k=4)) + '.jpg' #generate random image name
    filename = take_photo(f,0.8)                         #capture image
    print('Image {} - {}'.format(x+1,filename))

    image = cv2.imread(filename,cv2.IMREAD_COLOR)        #read captured image
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)       #convert image to grayscale

    faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml") 
    faces = faceCascade.detectMultiScale(                # Create Haar Cascade object and use Haar Cascade classifier                                        
          gray,                                          #.detect mutiscale generates a list of rectangles for all of the
          scaleFactor=1.3,                               # detected faces in the image
          minNeighbors=3,
          minSize=(20, 20)
  ) 

    for (x, y, w, h) in faces:                                    
      ROI = image[y:y+h, x:x+w]                          # Grab ROI (rectangle)
      blur = cv2.medianBlur(ROI, 65)                     # Apply blurring
      image[y:y+h, x:x+w] = blur                         # Insert ROI back into image

    
    plt.figure()                        
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

  except Exception as err:
    # Errors will be thrown if the user does not have a webcam or if they do not
    # grant the page permission to access it.
    print(str(err))